**Names of all group members:**
Nils Johnson, Matteo Bodmer, Hai-Yen Van, Lucas Gimeno, Jonas Isler

---

All code below is only suggestive and you may as well use different approaches.

In [2]:
# Exercise 1.
import numpy as np
np.random.seed(0)  # for reproducibility

# simulate explanatory variables x (the age and monthly income are assumed here to follow a continous uniform distribution)

# training data
m = 20000
# test data
n = 10000

# initialize data matrix
data = np.zeros((m+n,3))

# first feature of data is age between [18,80] sampled from uniform distribution
data[:,0] = np.random.uniform(18,80,(m+n))

# second feature of data is the monthly income in CHF 1000 in [1,15] sampled from uniform distribution
data[:,1] = np.random.uniform(1,15,(m+n))

# third feature is a binary probality of being slaraied / self-employed. Probability of being self employed is 10%
data[:,2] = np.random.choice([0,1],p=[0.9,0.1],size=(m+n))

# a) calculate empirical means and standard deviations over training data

mean_train_feature_1 = np.mean(data[:m,0])
mean_train_feature_2 = np.mean(data[:m,1])
mean_train_feature_3 = np.mean(data[:m,2])

std_train_feature_1 = np.std(data[:m,0])
std_train_feature_2 = np.std(data[:m,1])
std_train_feature_3 = np.std(data[:m,2])

# b) Can you come up with a few (2 or 3) additional features that may be relevant?
# (you don't have to implement those of course, just write down your answer in text)
#
#


In [3]:
# Exercise 2.
# Building the datasets:

sigmoid = lambda x: 1. / (1. + np.exp(-x))

x1, x2, x3 = data[:, 0], data[:, 1], data[:, 2]

p1 = sigmoid(13.3 - 0.33*x1 + 3.5*x2 - 3*x3)

# Indicator: 1 if x1 < 25 or x1 > 75, else 0
indicator = ((x1 < 25) | (x1 > 75)).astype(float)

p2 = sigmoid(5 - 10*indicator + 1.1*x2 - x3)

epsilon = np.random.uniform(size=(m+n))
# build the first dataset

y_1 = (p1 > epsilon).astype(int)

# build the second dataset
y_2 = (p2 > epsilon).astype(int)


In [4]:
# Exercise 2. a)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
# "model = LogisticRegression().fit(X_data, Y_data)" fits a model
# "pred_X = model.predict_proba(X)" evaluates the model
# (note that it outputs both P(Y=0|X) and P(Y=1|X))
# "log_loss(Y, pred_X)" evaluates the negative conditional log likelihood (also called cross-entropy loss)


# Fit the models on both datasets
model_1 = LogisticRegression().fit(data[:m,:],y_1[:m])
model_2 = LogisticRegression().fit(data[:m,:],y_2[:m])

# evaluate both models on the data, we only need the positive class to calculate the cross-entropy loss
pred_train_1 = model_1.predict_proba(data[:m,:])[:,1]
pred_train_2 = model_2.predict_proba(data[:m,:])[:,1]

pred_test_1 = model_1.predict_proba(data[m:,:])[:,1]
pred_test_2 = model_2.predict_proba(data[m:,:])[:,1]

# Calculate cross-entropy loss on both datasets for train and test
cross_entropy_train_1 = log_loss(y_1[:m], pred_train_1)
cross_entropy_train_2 = log_loss(y_2[:m], pred_train_2)

cross_entropy_test_1 = log_loss(y_1[m:], pred_test_1)
cross_entropy_test_2 = log_loss(y_2[m:], pred_test_2)

In [5]:
# Exercise 2. b)
# Calculate normalized data

data_norm = np.zeros(((m+n),3))

data_norm[:,0] = data[:,0]/std_train_feature_1
data_norm[:,1] = data[:,1]/std_train_feature_2
data_norm[:,2] = data[:,2]/std_train_feature_3

In [6]:
# Exercise 2.b) (i) and (ii)
from sklearn.svm import SVC
# "model = SVC(kernel='rbf', gamma=GAMMA, C=C, probability=True)" creates
# a model with kernel exp(-GAMMA \|x-x'\|_2^2) and regul. parameter C (note the relation between C and the parameter lambda from the lecture).
# "probability=True" enables the option "model.predict_proba(X)" to predict probabilities from the regression function \hat{f}^{svm}.
# "model.fit(X, Y)" optimizes the model parameters (using hinge loss)

# Fit the models for both datasets (this can take up to 60 seconds with SVC)
model_svm_1 = SVC(kernel='rbf', gamma = 0.1, C= 0.2, probability=True).fit(data_norm[:m,:],y_1[:m])
model_svm_2 = SVC(kernel='rbf', gamma= 0.1, C=0.2, probability=True).fit(data_norm[:m,:],y_2[:m])


In [7]:
# Exercise 2.b (iii)
# "model.predict_proba(X)" predicts probabilities from features (note that it outputs both P(Y=0|X) and P(Y=1|X))
pred_svm_train_1 = model_svm_1.predict_proba(data_norm[:m,:])[:,1]
pred_svm_train_2 = model_svm_2.predict_proba(data_norm[:m,:])[:,1]

pred_svm_test_1 = model_svm_1.predict_proba(data_norm[m:,:])[:,1]
pred_svm_test_2 = model_svm_2.predict_proba(data_norm[m:,:])[:,1]

# Calculate cross-entropy loss on both normalized datasets for train and test
cross_entropy_svm_train_1 = log_loss(y_1[:m], pred_svm_train_1)
cross_entropy_svm_train_2 = log_loss(y_2[:m], pred_svm_train_2)

cross_entropy_svm_test_1 = log_loss(y_1[m:], pred_svm_test_1)
cross_entropy_svm_test_2 = log_loss(y_2[m:], pred_svm_test_2)

In [ ]:
# Exercise 2.b (iv)
#Would the results change using standardized data instead of normalized data? No need to run any code here, only explain your reasoning.

In [ ]:
# Exercise 2.c
import matplotlib.pyplot as plt
# To calculate the curves, it is fine to take 100 threshold values c, i.e.,
ths = np.linspace(0, 1, 100)

# To approximately calculate the AUC, it is fine to simply use Riemann sums.
# This means, if you have 100 (a_i, b_i) pairs for the curves, a_1 <= a_2 <= ...
# then you may simply use the sum
# sum_{i=1}^99 (b_i + b_{i+1})/2 * (a_{i+1}-a_i)
# as the approximation of the integral (or AUC)


# first data set & logistic regression:
# (the code should be reusable for all cases, only exchanging datasets and predicted probabilities depending on the model)

# Calculate positives (only depending on the dataset)

# Calculate true positives for all threshold values

# Calculate false positives for all threshold values

# Calculate FDR and TPR rate (points on the FDR/TPR curve) and the AUC


# second data set & logistic regression:


# first data set and rkhs regression:


# second data set and rkhs regression:

In [ ]:
# Exercise 3.

# Set model parameters and define matrix D


# Scenario 1:
# Define Portfolio and possible outcomes for this portfolio using matrix D


# Plot histogram of profits and losses


# Calculate expected profit and losses and 95%-VaR


# Scenario 2:
# Define Portfolio and possible outcomes using the matrix D and the predicted default probabilities from the logistic regression model


# Plot histogram of profits and losses


# Calculate expected profit and losses and 95%-VaR


# Scenario 3:
# Define Portfolio and possible outcomes using the matrix D and the predicted default probabilities from the rkhs model


# Plot histogram of profits & losses (which is simply the performance of each strategy in scenario k, i.e. a profit if positive or a loss if negative)


# Calculate expected profit & losses and 95%-VaR for each strategy